<center>
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <h3><b>Objective</b></h3>
    <b>Get publicly accessible amazon RDS instances using unSkript actions.</b>
</div>

<br>
</center>
<center><h2>Publicly Accessible Amazon RDS Instances</h2></center>

# Steps Overview
   1)[List Publicly Accessible RDS Instances](#1)</br>

### AWS List All Regions
In this action, we shall use user given region list if not provided we generate all region AWS list.

>Output variable: `region`

In [4]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')
task.configure(outputName="region")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

### <a id="1">List Publicly Accessible RDS Instances</a>
Using unSkript's <b>AWS List Publicly Accessible RDS Instances</b> action, we will fetch all the publicly accessible instances from the list of DB instances.

>Input parameters: `region`

>Output variable: `rds_instances`

In [5]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, List, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_publicly_accessible_db_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_publicly_accessible_db_instances(handle, region: str = "") -> Tuple:
    """aws_publicly_accessible_db_instances Gets all Publicly accessible DBs

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the RDS.

        :rtype: List with publicly accessible RDS instances.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            ec2Client = handle.client('rds', region_name=reg)
            response = aws_get_paginator(ec2Client, "describe_db_instances", "DBInstances")
            for db in response:
                db_instance_dict = {}
                if db['PubliclyAccessible']:
                    db_instance_dict["region"] = reg
                    db_instance_dict["instance"] = db['DBInstanceIdentifier']
                    result.append(db_instance_dict)
        except Exception as error:
            pass
    execution_flag = False
    if len(result) > 0:
        execution_flag = True
    output = (execution_flag, result)
    return output


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(outputName="rds_instances")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_publicly_accessible_db_instances, lego_printer=aws_publicly_accessible_db_instances_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS actions. This runbook help to find Publicly Accessible RDS Instances. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io